<a href="https://colab.research.google.com/github/alice410451027/QA-System-with-Retrieval-Augmented-Generation-RAG/blob/main/Q%26A_System_with_Retrieval_Augmented_Generation_(RAG).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Q&A System with Retrieval-Augmented Generation (RAG)
1. First download the vectorized zip file and upload it to Google Drive.
2. Set the folder access permissions to 'Anyone with the link can view.
3. Then copy the link to Google Drive Direct Link for conversion. https://sites.google.com/site/gdocs2direct/

In [1]:
URL = "https://drive.google.com/uc?export=download&id=1YjR-2k7TYusci1_qHzSMk7LAsEzWqv-m"

!wget -O faiss_db.zip "$URL"

--2025-04-21 05:55:14--  https://drive.google.com/uc?export=download&id=1YjR-2k7TYusci1_qHzSMk7LAsEzWqv-m
Resolving drive.google.com (drive.google.com)... 173.194.206.102, 173.194.206.138, 173.194.206.101, ...
Connecting to drive.google.com (drive.google.com)|173.194.206.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1YjR-2k7TYusci1_qHzSMk7LAsEzWqv-m&export=download [following]
--2025-04-21 05:55:14--  https://drive.usercontent.google.com/download?id=1YjR-2k7TYusci1_qHzSMk7LAsEzWqv-m&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.126.132, 2607:f8b0:4001:c1d::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.126.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 115934 (113K) [application/octet-stream]
Saving to: ‘faiss_db.zip’

faiss_db.zip        100%[===================>] 113.22K  --.-KB/

In [2]:
!pip install -U gdown

import gdown
file_id = "1FLXF9JB6owbxtUDwAVzj_PHawYAuhjae"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "faiss_db.zip", quiet=False)

!unzip -o faiss_db.zip -d faiss_db/

Downloading...
From (original): https://drive.google.com/uc?id=1FLXF9JB6owbxtUDwAVzj_PHawYAuhjae
From (redirected): https://drive.google.com/uc?id=1FLXF9JB6owbxtUDwAVzj_PHawYAuhjae&confirm=t&uuid=cb4e5993-7f6d-4f33-b783-7511bb8f5963
To: /content/faiss_db.zip
100%|██████████| 51.0M/51.0M [00:00<00:00, 60.4MB/s]


Archive:  faiss_db.zip
   creating: faiss_db/faiss_db/
  inflating: faiss_db/faiss_db/index.faiss  
  inflating: faiss_db/faiss_db/index.pkl  


In [3]:
!unzip faiss_db.zip

Archive:  faiss_db.zip
  inflating: faiss_db/index.faiss    
  inflating: faiss_db/index.pkl      


### 1. Install Required Packages

In [4]:
!pip install -U langchain langchain-community sentence-transformers faiss-cpu gradio openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 114.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/8

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

In [6]:
from openai import OpenAI
import gradio as gr

### 2. Custom E5 Embedding

In [7]:
class CustomE5Embedding(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        return super().embed_query(f"query: {text}")

### 3. Load FAISS Vectorstore

In [8]:
embedding_model = CustomE5Embedding(model_name="intfloat/multilingual-e5-small")
db = FAISS.load_local("faiss_db", embedding_model, allow_dangerous_deserialization=True)
retriever = db.as_retriever()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/498k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

### 4. Set Up LLM

In [9]:
import os
from google.colab import userdata

In [10]:
api_key = userdata.get('GenAI_groq')

In [11]:
os.environ["OPENAI_API_KEY"] = api_key

In [12]:
model = "llama3-70b-8192"
base_url="https://api.groq.com/openai/v1"

In [13]:
client = OpenAI(
    base_url=base_url
)

### 5. Prompt Design

In [15]:
system_prompt = "You are an expert in academic writing, specializing in APA 7th edition formatting. Your role is to assist students and beginners in understanding and applying APA style rules accurately and clearly. Please explain with patience and clarity, and when appropriate, include practical examples to help the user grasp the concept."

prompt_template = """
Use the following reference material to answer the user's question:
{retrieved_chunks}

The user's question is: {question}

If the answer cannot be found in the provided material, politely let the user know and suggest checking the official APA Manual or the APA Style website for more details.
"""


### 6. RAG Function

In [18]:
chat_history = []

def chat_with_rag(user_input):
    global chat_history

    docs = retriever.get_relevant_documents(user_input)
    retrieved_chunks = "\n\n".join([doc.page_content for doc in docs])

    final_prompt = prompt_template.format(retrieved_chunks=retrieved_chunks, question=user_input)

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": final_prompt},
        ]
    )
    answer = response.choices[0].message.content

    chat_history.append((user_input, answer))
    return answer


### 7. Build Web App with Gradio

In [19]:
with gr.Blocks() as demo:
    gr.Markdown("# 📘 APA 7 Writing Assistant")
    gr.Markdown("Ask any question about APA 7th edition formatting rules, and get clear guidance based on official documentation.")

    chatbot = gr.Chatbot(label="APA Assistant")
    msg = gr.Textbox(placeholder="Type your APA formatting question here...")

    def respond(message, chat_history_local):
        response = chat_with_rag(message)
        chat_history_local.append((message, response))
        return "", chat_history_local

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(debug=True)


<ipython-input-19-bfe68359c371>:5: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="APA Assistant")


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7814fe275ed8c22fa2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


<ipython-input-18-e6603e8c00f6>:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(user_input)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://7814fe275ed8c22fa2.gradio.live
